# Graph Build + Routing Code

### Import neccessary libaries

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, Point
from collections import defaultdict

import random
import math
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Sequence, Tuple, Union



## Read in the data

In [2]:
junction_df = gpd.read_parquet("../data/merged/berlin_bike_accident_node_panel.parquet")
acc_node_df = gpd.read_parquet("../data/merged/acc_node.parquet")

# Ensure expected CRS for folium (lat/lon)
junction_df = junction_df.to_crs(epsg=4326)
acc_node_df = acc_node_df.to_crs(epsg=4326)

# If these columns aren't present yet, create them from geometry
if "longitude" not in junction_df.columns:
    junction_df["longitude"] = junction_df.geometry.x
if "latitude" not in junction_df.columns:
    junction_df["latitude"] = junction_df.geometry.y

if "longitude" not in acc_node_df.columns:
    acc_node_df["longitude"] = acc_node_df.geometry.x
if "latitude" not in acc_node_df.columns:
    acc_node_df["latitude"] = acc_node_df.geometry.y

# Backwards compatibility: if you didn't add has_crossing, derive it
if "has_crossing" not in acc_node_df.columns:
    acc_node_df["has_crossing"] = acc_node_df["node_id"].notna()


In [3]:
segments_df = gpd.read_parquet("../data/merged/berlin_bike_accident_strava_risk_core_panel.parquet").to_crs(epsg=4326)

## Prepare data

In [4]:
# one geometry per node_id
crossings_gdf = (
    junction_df[["node_id", "geometry"]]
    .dropna(subset=["node_id", "geometry"])
    .drop_duplicates(subset=["node_id"])
    .copy()
)

crossings_gdf = gpd.GeoDataFrame(
    crossings_gdf,
    geometry="geometry",
    crs=junction_df.crs,
)

print("crossings_gdf:", crossings_gdf.shape)


crossings_gdf: (2924, 2)


In [5]:
rep = segments_df.geometry.representative_point()
segments_df["longitude"] = rep.x
segments_df["latitude"] = rep.y

In [6]:
coords_to_segments = defaultdict(set)

for name, geom in zip(segments_df["counter_name"], segments_df.geometry):
    for lon, lat in geom.coords:
        coords_to_segments[(lat, lon)].add(name)

# Graph + routing

Below is a **clean, paper- and repo-friendly Markdown description** you can drop into a README, notebook markdown cell, or a `docs/` page. It’s concise, structured, and matches exactly what the code does—no implementation noise, but no hand-waving either.

---

## Risk-aware routing graph: overview and structure

This module constructs a **month-specific routing graph** from street segments, attaches **exposure-normalized crash risk** at both the segment and junction level, and provides utilities for **shortest-path** and **safety-aware routing**.

At a high level, the pipeline is:

> **Monthly risk panels → routing graph → edge costs → routing queries**

---

## Helpers

### Node representation

Graph nodes are represented as `(x, y)` coordinate tuples in a **metric CRS** (UTM). Nodes correspond to **segment endpoints**, not pre-defined network IDs.

### Stable endpoint extraction

`_edge_endpoints_as_node_keys` converts a segment LineString into two node keys by:

* taking the first and last coordinates,
* rounding them (typically to 1 m) to ensure adjacent segments share identical nodes.

This makes the graph robust to floating-point and digitization noise.

### Risk fallback handling

`_risk_fallback_value` defines how missing (`NaN`) risk values are handled:

* default: **median** of observed risks in the same month,
* alternatives: mean, high (90th percentile), or zero.

This avoids treating missing risk as “safe” while keeping routing numerically stable.

### Junction risk lookup

`_build_node_risk_lookup` constructs a dictionary
`node_id → junction risk`
for a given `(year, month)` from the junction panel. It:

* filters to the month,
* ensures one value per junction,
* fills missing risks using the fallback policy.

### Mapping junctions to graph nodes

`_attach_node_ids_to_graph_nodes` spatially **snaps junction points** to the nearest graph node (within a tolerance).
This step attaches `node_id` metadata to graph nodes and is **required** for junction risk to influence routing.

### Unified edge iteration

`_iter_edges` and `_get_edge_data_for_step` abstract over `Graph` vs `MultiGraph` so downstream code can:

* iterate edges uniformly,
* select the correct parallel edge when summarizing routes.

---

## Graph construction

### Configuration

`GraphBuildConfig` centralizes all graph-building decisions:

* CRS and endpoint rounding,
* exposure and risk column selection,
* handling of zero-exposure segments,
* fallback policy for missing risk,
* whether to allow parallel edges.

This makes the graph construction reproducible and schema-robust.

### Monthly graph build

`build_routing_graph_for_month` constructs a routing graph for a specific `(year, month)`:

1. Filter segment panel to the month.
2. Project to metric CRS for correct distances.
3. Identify exposure and


In [7]:
# This code builds a routing graph for a given (year, month), attaches risk,
# and provides shortest / constrained-min-risk routing utilities.


# Helpers

NodeKey = Tuple[float, float]


def _edge_endpoints_as_node_keys(linestring: LineString, ndigits: int = 0) -> Tuple[NodeKey, NodeKey]:
    """
    Create stable node keys from LineString endpoints in a metric CRS.

    For UTM meters:
      ndigits=0 => 1m rounding (robust)
      ndigits=1 => 0.1m rounding (less robust)
      ndigits=3 => 1mm rounding (usually too strict)
    """
    (x1, y1) = linestring.coords[0]
    (x2, y2) = linestring.coords[-1]
    a = (round(float(x1), ndigits), round(float(y1), ndigits))
    b = (round(float(x2), ndigits), round(float(y2), ndigits))
    return a, b


def _risk_fallback_value(series: pd.Series, *, strategy: str = "median", default: float = 0.0) -> float:
    """
    Decide a safe fallback risk if a segment/node risk is NaN.

    strategy:
      - "median": median of observed values for that month panel
      - "mean": mean of observed values
      - "zero": always 0
      - "high": conservative high value (90th percentile if possible)
    """
    s = pd.to_numeric(series, errors="coerce")
    s = s[np.isfinite(s)]
    if len(s) == 0:
        return float(default)

    if strategy == "median":
        return float(np.nanmedian(s))
    if strategy == "mean":
        return float(np.nanmean(s))
    if strategy == "high":
        return float(np.nanpercentile(s, 90))
    if strategy == "zero":
        return 0.0

    return float(np.nanmedian(s))


def _build_node_risk_lookup(
    junction_panel_gdf: gpd.GeoDataFrame,
    year: int,
    month: int,
    *,
    node_id_col: str = "node_id",
    node_risk_col: str = "risk_accidents_per_10k_trips",
    fallback: float = 0.0,
    fallback_strategy: str = "median",
) -> Dict[str, float]:
    """
    Returns dict[node_id(str)] -> node_risk for the given (year, month).

    Notes:
      - If (year, month) missing entirely: returns {} (caller chooses behavior).
      - If a node risk value is NaN: filled by fallback computed from the month panel.
    """
    if junction_panel_gdf is None or len(junction_panel_gdf) == 0:
        return {}

    j = junction_panel_gdf[(junction_panel_gdf["year"] == year) & (junction_panel_gdf["month"] == month)].copy()
    if j.empty:
        return {}

    if node_risk_col not in j.columns:
        raise KeyError(f"Missing node risk column '{node_risk_col}' in junction_panel_gdf")

    # Ensure one row per node_id-month; if multiple exist, take mean (shouldn't happen, but safe).
    j = j.groupby(node_id_col, as_index=False)[node_risk_col].mean()

    fb = _risk_fallback_value(j[node_risk_col], strategy=fallback_strategy, default=fallback)
    j[node_risk_col] = pd.to_numeric(j[node_risk_col], errors="coerce").fillna(fb)

    return dict(zip(j[node_id_col].astype(str), j[node_risk_col].astype(float)))


def _attach_node_ids_to_graph_nodes(
    G: nx.Graph,
    crossings_gdf: gpd.GeoDataFrame,
    *,
    metric_epsg: int = 32633,
    node_id_col: str = "node_id",
    max_snap_m: float = 20.0,
) -> None:
    """
    Map your existing 'node_id' (junctions/crossings) onto graph nodes.

    We snap each crossing point to the nearest graph node (within max_snap_m).
    Stores: G.nodes[node_key]["node_id"] = <id>

    Critical for applying junction risk penalties.
    """
    if crossings_gdf is None or len(crossings_gdf) == 0:
        return

    crossings_m = crossings_gdf.to_crs(epsg=metric_epsg).copy()

    graph_nodes = list(G.nodes())
    if len(graph_nodes) == 0:
        return

    graph_xy = np.array(graph_nodes, dtype=float)

    for _, row in crossings_m.iterrows():
        if row.geometry is None or row.geometry.is_empty:
            continue

        nid = str(row[node_id_col])
        x, y = float(row.geometry.x), float(row.geometry.y)

        d2 = (graph_xy[:, 0] - x) ** 2 + (graph_xy[:, 1] - y) ** 2
        j = int(d2.argmin())
        dist = float(math.sqrt(d2[j]))
        if dist <= max_snap_m:
            node_key = graph_nodes[j]
            G.nodes[node_key]["node_id"] = nid


def _iter_edges(G: nx.Graph):
    """
    Unified iterator over edges returning (u, v, key, data) where key is None for non-multigraphs.
    """
    if isinstance(G, nx.MultiGraph):
        for u, v, k, data in G.edges(keys=True, data=True):
            yield u, v, k, data
    else:
        for u, v, data in G.edges(data=True):
            yield u, v, None, data


def _get_edge_data_for_step(G: nx.Graph, a, b, *, weight_key: str = "cost") -> dict:
    """
    For MultiGraph: pick the edge (between a,b) that minimizes weight_key, fallback to length_m.
    For Graph: return edge data.
    """
    if isinstance(G, nx.MultiGraph):
        edges = G.get_edge_data(a, b)
        if edges is None:
            raise KeyError(f"No edge between {a} and {b}")

        best = None
        best_val = None
        for _, d in edges.items():
            val = d.get(weight_key, d.get("length_m", 0.0))
            if best is None or val < best_val:
                best = d
                best_val = val
        return best
    else:
        d = G.get_edge_data(a, b)
        if d is None:
            raise KeyError(f"No edge between {a} and {b}")
        return d


# Graph construction

@dataclass(frozen=True)
class GraphBuildConfig:
    metric_epsg: int = 32633
    endpoint_ndigits: int = 0              # 0 => 1m rounding
    seg_id_col: str = "counter_name"
    seg_exposure_col_candidates: Tuple[str, ...] = ("monthly_strava_trips", "sum_strava_total_trip_count")
    seg_risk_col_candidates: Tuple[str, ...] = ("risk_accidents_per_10k_trips", "risk_accidents_per_trip")
    # if using per-trip risk, we scale to per-10k automatically for routing:
    auto_scale_trip_risk_to_per_10k: bool = True
    # risk fallback policy if risk is NaN:
    risk_fallback_strategy: str = "median"   # median/mean/high/zero
    risk_fallback_default: float = 0.0
    # when exposure is 0 / missing:
    drop_edges_with_zero_exposure: bool = True
    keep_parallel_edges: bool = True


def build_routing_graph_for_month(
    segments_panel_gdf: gpd.GeoDataFrame,
    year: int,
    month: int,
    *,
    config: GraphBuildConfig = GraphBuildConfig(),
) -> nx.Graph:
    """
    Build a routing graph for one (year, month) from segment LineStrings.

    Nodes: segment endpoints (rounded in metric CRS).
    Edges carry:
      - segment_id
      - length_m
      - seg_risk (scaled for routing; typically per-10k trips)
      - geometry

    Important fixes vs the earlier version:
      - avoids mixing units by preferring per-10k risk; auto-scales per-trip risk if needed
      - optionally drops edges with zero exposure for that month (safer than treating unknown as 0)
      - robust fallback for missing risk (median/mean/high)
    """
    df = segments_panel_gdf[(segments_panel_gdf["year"] == year) & (segments_panel_gdf["month"] == month)].copy()
    if df.empty:
        raise ValueError(f"No segment rows found for year={year}, month={month}")

    df_m = df.to_crs(epsg=config.metric_epsg).copy()

    # Pick exposure column
    exposure_col = None
    for c in config.seg_exposure_col_candidates:
        if c in df_m.columns:
            exposure_col = c
            break
    if exposure_col is None:
        raise KeyError(f"Missing exposure column. Tried: {config.seg_exposure_col_candidates}")

    # Pick risk column
    risk_col = None
    for c in config.seg_risk_col_candidates:
        if c in df_m.columns:
            risk_col = c
            break
    if risk_col is None:
        raise KeyError(f"Missing risk column. Tried: {config.seg_risk_col_candidates}")

    # Ensure uniqueness at segment-month level
    if df_m[[config.seg_id_col, "year", "month"]].duplicated().any():
        # This should not happen in a proper panel; fail loudly to avoid silently building duplicates
        dups = int(df_m[[config.seg_id_col, "year", "month"]].duplicated().sum())
        raise ValueError(f"segments_panel_gdf has {dups} duplicate rows for (segment,year,month). Fix upstream aggregation.")

    # Optionally drop edges with zero exposure (or missing exposure)
    df_m[exposure_col] = pd.to_numeric(df_m[exposure_col], errors="coerce")
    if config.drop_edges_with_zero_exposure:
        df_m = df_m[df_m[exposure_col].fillna(0) > 0].copy()

    # Compute fallback risk (month-specific)
    fb = _risk_fallback_value(df_m[risk_col], strategy=config.risk_fallback_strategy, default=config.risk_fallback_default)

    # Build graph
    G = nx.MultiGraph() if config.keep_parallel_edges else nx.Graph()

    for _, row in df_m.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue
        if not isinstance(geom, LineString):
            # If MultiLineString, take the longest component (common safe fallback)
            try:
                parts = list(geom.geoms)
                parts = [p for p in parts if isinstance(p, LineString) and not p.is_empty]
                if not parts:
                    continue
                geom = max(parts, key=lambda g: g.length)
            except Exception:
                continue

        u, v = _edge_endpoints_as_node_keys(geom, ndigits=config.endpoint_ndigits)
        seg_id = str(row[config.seg_id_col])
        length_m = float(geom.length)

        seg_risk = pd.to_numeric(row[risk_col], errors="coerce")
        if pd.isna(seg_risk):
            seg_risk = fb
        seg_risk = float(seg_risk)

        # If only per-trip risk exists, scale to per-10k for routing stability
        if (risk_col == "risk_accidents_per_trip") and config.auto_scale_trip_risk_to_per_10k:
            seg_risk *= 10_000.0

        if u not in G:
            G.add_node(u, x=u[0], y=u[1])
        if v not in G:
            G.add_node(v, x=v[0], y=v[1])

        G.add_edge(
            u,
            v,
            segment_id=seg_id,
            length_m=length_m,
            seg_risk=seg_risk,
            geometry=geom,
        )

    if G.number_of_edges() == 0:
        raise ValueError(f"Graph has 0 edges for year={year}, month={month} after filtering. Check exposure coverage and filters.")

    return G


# Cost attributes (routing objective)

@dataclass(frozen=True)
class CostConfig:
    alpha: float = 1.0   # weight on length (meters)
    beta: float = 1.0    # weight on segment risk (expected to be per-10k scale)
    gamma: float = 0.0   # weight on junction/node risk (optional; expected per-10k scale)


def add_cost_attributes(
    G: nx.Graph,
    *,
    cost_cfg: CostConfig = CostConfig(),
    node_risk_by_node_id: Optional[Dict[str, float]] = None,
) -> None:
    """
    Adds 'cost' and 'node_penalty' to each edge.

    cost = alpha*length_m + beta*seg_risk + gamma*node_penalty

    node_penalty is an edge-local approximation:
      node_penalty = 0.5*(risk(node_u)+risk(node_v))

    Fixes vs earlier version:
      - correct iteration for Graph vs MultiGraph
      - safe default for missing node ids
      - stores into edge dict consistently
    """
    if node_risk_by_node_id is None:
        node_risk_by_node_id = {}

    for u, v, k, data in _iter_edges(G):
        length_m = float(data.get("length_m", 0.0))
        seg_risk = float(data.get("seg_risk", 0.0))

        node_penalty = 0.0
        if cost_cfg.gamma != 0.0:
            uid = G.nodes[u].get("node_id")
            vid = G.nodes[v].get("node_id")
            ru = float(node_risk_by_node_id.get(uid, 0.0)) if uid is not None else 0.0
            rv = float(node_risk_by_node_id.get(vid, 0.0)) if vid is not None else 0.0
            node_penalty = 0.5 * (ru + rv)

        cost = cost_cfg.alpha * length_m + cost_cfg.beta * seg_risk + cost_cfg.gamma * node_penalty

        if isinstance(G, nx.MultiGraph):
            G.edges[u, v, k]["node_penalty"] = float(node_penalty)
            G.edges[u, v, k]["cost"] = float(cost)
        else:
            data["node_penalty"] = float(node_penalty)
            data["cost"] = float(cost)


# Routing utilities

def nearest_graph_node(
    G: nx.Graph,
    lon: float,
    lat: float,
    *,
    metric_epsg: int = 32633,
) -> NodeKey:
    """
    Snap an (lon, lat) point to the closest graph node.
    Brute force over nodes (OK for moderate graphs). Replace with KDTree if needed.
    """
    p = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326").to_crs(epsg=metric_epsg).iloc[0]
    x, y = float(p.x), float(p.y)

    nodes = list(G.nodes())
    if len(nodes) == 0:
        raise ValueError("Graph has no nodes")

    xy = np.array(nodes, dtype=float)
    d2 = (xy[:, 0] - x) ** 2 + (xy[:, 1] - y) ** 2
    idx = int(d2.argmin())
    return nodes[idx]


def route_stats(G: nx.Graph, path: List[NodeKey]) -> Dict[str, float]:
    """
    Summarize route length and risk along a path of nodes.

    For MultiGraph, uses the minimum-'cost' edge between consecutive nodes if present.
    """
    total_len = 0.0
    total_seg_risk = 0.0
    total_cost = 0.0
    total_node_penalty = 0.0

    for a, b in zip(path[:-1], path[1:]):
        d = _get_edge_data_for_step(G, a, b, weight_key="cost")

        total_len += float(d.get("length_m", 0.0))
        total_seg_risk += float(d.get("seg_risk", 0.0))
        total_node_penalty += float(d.get("node_penalty", 0.0))
        total_cost += float(d.get("cost", 0.0))

    return {
        "length_m": total_len,
        "seg_risk_sum": total_seg_risk,
        "node_penalty_sum": total_node_penalty,
        "cost_sum": total_cost,
    }


def shortest_path_by(G: nx.Graph, source: NodeKey, target: NodeKey, weight: str) -> List[NodeKey]:
    return nx.shortest_path(G, source=source, target=target, weight=weight, method="dijkstra")


def constrained_min_risk_route(
    G: nx.Graph,
    source: NodeKey,
    target: NodeKey,
    *,
    eps: float = 0.10,                 # allow up to +10% distance over shortest
    length_attr: str = "length_m",
    risk_attr: str = "seg_risk",
    lambdas: Optional[List[float]] = None,
) -> List[NodeKey]:
    """
    Minimize risk subject to length <= (1+eps) * shortest_length.

    Implementation: parametric sweep with a Lagrangian:
      minimize (risk + lambda * length)
    and select the best path that satisfies the length constraint.

    Fixes vs earlier version:
      - correct edge iteration for Graph vs MultiGraph
      - uses actual per-edge length/risk attributes (not route_stats hardcoded)
      - does not rely on 'cost' existing
    """
    shortest_len_path = shortest_path_by(G, source, target, weight=length_attr)
    shortest_len = route_stats(G, shortest_len_path)["length_m"]
    max_len = (1.0 + eps) * shortest_len

    if lambdas is None:
        lambdas = [0.0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0, 1000.0]

    best_path = None
    best_risk = None
    best_len = None

    # Build temporary combined weight on edges: risk + lambda*length
    for lam in lambdas:
        for u, v, k, d in _iter_edges(G):
            comb = float(d.get(risk_attr, 0.0)) + lam * float(d.get(length_attr, 0.0))
            if isinstance(G, nx.MultiGraph):
                G.edges[u, v, k]["_comb"] = comb
            else:
                d["_comb"] = comb

        p = shortest_path_by(G, source, target, weight="_comb")
        st = route_stats(G, p)
        if st["length_m"] <= max_len:
            if best_path is None or st["seg_risk_sum"] < best_risk or (st["seg_risk_sum"] == best_risk and st["length_m"] < best_len):
                best_path = p
                best_risk = st["seg_risk_sum"]
                best_len = st["length_m"]

    return best_path if best_path is not None else shortest_len_path


# End-to-end wrapper

@dataclass(frozen=True)
class RoutingMonthArtifacts:
    G: nx.Graph
    year: int
    month: int
    segment_risk_col_used: str
    node_risk_col_used: Optional[str]
    notes: str


def build_graph_with_costs_for_month(
    segments_panel_gdf: gpd.GeoDataFrame,
    year: int,
    month: int,
    *,
    crossings_gdf: Optional[gpd.GeoDataFrame] = None,
    junction_panel_gdf: Optional[gpd.GeoDataFrame] = None,
    graph_cfg: GraphBuildConfig = GraphBuildConfig(),
    cost_cfg: CostConfig = CostConfig(alpha=1.0, beta=1.0, gamma=0.0),
    node_risk_col: str = "risk_accidents_per_10k_trips",
    node_snap_m: float = 30.0,
    node_risk_fallback_strategy: str = "median",
    node_risk_fallback_default: float = 0.0,
) -> RoutingMonthArtifacts:
    """
    Build the month-specific routing graph and attach cost attributes.
    This is the piece that makes the routing claim in your paper reproducible.

    What it does:
      1) Build graph edges from segment panel for (year, month)
      2) Optionally attach node_ids to graph nodes by snapping crossings_gdf points
      3) Optionally build node risk lookup from junction_panel_gdf for (year, month)
      4) Add edge costs combining length + segment risk + optional junction penalties

    Critical: This is where junction risk actually gets integrated (if gamma != 0).
    """
    G = build_routing_graph_for_month(segments_panel_gdf, year, month, config=graph_cfg)

    node_risk_lookup = {}
    used_node_risk_col = None
    notes = []

    if cost_cfg.gamma != 0.0:
        if crossings_gdf is None or junction_panel_gdf is None:
            notes.append("gamma!=0 but crossings_gdf/junction_panel_gdf not provided -> junction penalty disabled.")
        else:
            _attach_node_ids_to_graph_nodes(
                G,
                crossings_gdf,
                metric_epsg=graph_cfg.metric_epsg,
                node_id_col="node_id",
                max_snap_m=node_snap_m,
            )
            node_risk_lookup = _build_node_risk_lookup(
                junction_panel_gdf,
                year,
                month,
                node_id_col="node_id",
                node_risk_col=node_risk_col,
                fallback=node_risk_fallback_default,
                fallback_strategy=node_risk_fallback_strategy,
            )
            used_node_risk_col = node_risk_col

            attached = sum(1 for n in G.nodes if "node_id" in G.nodes[n])
            notes.append(f"Attached node_id to {attached}/{G.number_of_nodes()} graph nodes (snap<= {node_snap_m}m).")

    add_cost_attributes(G, cost_cfg=cost_cfg, node_risk_by_node_id=node_risk_lookup)

    # Determine which segment risk column was used (for transparency)
    seg_risk_col_used = None
    for c in graph_cfg.seg_risk_col_candidates:
        if c in segments_panel_gdf.columns:
            seg_risk_col_used = c
            break
    if seg_risk_col_used is None:
        seg_risk_col_used = "(unknown)"

    # If per-trip risk was used but scaled, note it
    if seg_risk_col_used == "risk_accidents_per_trip" and graph_cfg.auto_scale_trip_risk_to_per_10k:
        notes.append("Scaled segment risk_accidents_per_trip by 10,000 for routing stability (per-10k trips).")

    return RoutingMonthArtifacts(
        G=G,
        year=year,
        month=month,
        segment_risk_col_used=seg_risk_col_used,
        node_risk_col_used=used_node_risk_col,
        notes=" ".join(notes).strip(),
    )


# Verification utilities

def verify_graph_sanity(
    artifacts: RoutingMonthArtifacts,
    *,
    expect_junction_penalties: bool = False,
) -> Dict[str, Union[int, float, str]]:
    """
    Quick, paper-friendly sanity checks:
      - graph non-empty
      - risk magnitude reasonable
      - cost depends on risk (not strictly provable here, but we report ranges)
      - node ids attached when expected
    """
    G = artifacts.G

    seg_risks = [float(d.get("seg_risk", 0.0)) for _, _, _, d in _iter_edges(G)]
    lengths = [float(d.get("length_m", 0.0)) for _, _, _, d in _iter_edges(G)]
    costs = [float(d.get("cost", 0.0)) for _, _, _, d in _iter_edges(G)]
    node_pen = [float(d.get("node_penalty", 0.0)) for _, _, _, d in _iter_edges(G)]

    attached_nodes = sum(1 for n in G.nodes if "node_id" in G.nodes[n])

    out = {
        "n_nodes": G.number_of_nodes(),
        "n_edges": G.number_of_edges(),
        "seg_risk_min": float(np.min(seg_risks)) if seg_risks else float("nan"),
        "seg_risk_median": float(np.median(seg_risks)) if seg_risks else float("nan"),
        "seg_risk_max": float(np.max(seg_risks)) if seg_risks else float("nan"),
        "length_median": float(np.median(lengths)) if lengths else float("nan"),
        "cost_median": float(np.median(costs)) if costs else float("nan"),
        "node_penalty_nonzero_share": float(np.mean(np.array(node_pen) > 0)) if node_pen else 0.0,
        "graph_node_ids_attached": attached_nodes,
        "notes": artifacts.notes,
    }

    if expect_junction_penalties and attached_nodes == 0:
        out["warning"] = "Expected junction penalties, but no graph nodes have node_id attached. Check snapping tolerance and CRS."
    return out


## Run routing for one OD pair

In [8]:
# Run routing for ONE OD pair

def run_one_od_routing(
    *,
    segments_panel_gdf: gpd.GeoDataFrame,
    crossings_gdf: gpd.GeoDataFrame,
    junction_panel_gdf: gpd.GeoDataFrame,
    year: int,
    month: int,
    origin_lonlat: tuple[float, float],
    dest_lonlat: tuple[float, float],
    eps: float = 0.10,
    # cost weights
    alpha: float = 1.0,
    beta: float = 1.0,
    gamma: float = 0.0,
    metric_epsg: int = 32633,
):
    """
    Builds the month graph, snaps OD to graph nodes, and returns:
      - shortest-by-length route
      - constrained min-risk route within (1+eps) distance

    origin_lonlat/dest_lonlat are (lon, lat) in EPSG:4326.
    """
    artifacts = build_graph_with_costs_for_month(
        segments_panel_gdf,
        year,
        month,
        crossings_gdf=crossings_gdf,
        junction_panel_gdf=junction_panel_gdf,
        graph_cfg=GraphBuildConfig(metric_epsg=metric_epsg),
        cost_cfg=CostConfig(alpha=alpha, beta=beta, gamma=gamma),
        node_snap_m=20.0,
    )

    sanity = verify_graph_sanity(artifacts, expect_junction_penalties=(gamma != 0.0))

    G = artifacts.G
    o_lon, o_lat = origin_lonlat
    d_lon, d_lat = dest_lonlat

    src = nearest_graph_node(G, o_lon, o_lat, metric_epsg=metric_epsg)
    dst = nearest_graph_node(G, d_lon, d_lat, metric_epsg=metric_epsg)

    # Baseline: shortest distance
    p_len = shortest_path_by(G, src, dst, weight="length_m")
    st_len = route_stats(G, p_len)

    # Constrained min-risk (risk attribute is 'seg_risk' on edges)
    p_safe = constrained_min_risk_route(G, src, dst, eps=eps, length_attr="length_m", risk_attr="seg_risk")
    st_safe = route_stats(G, p_safe)

    # Also compute shortest-by-cost if you want (mixed objective)
    p_cost = shortest_path_by(G, src, dst, weight="cost")
    st_cost = route_stats(G, p_cost)

    return {
        "graph_sanity": sanity,
        "origin_node": src,
        "dest_node": dst,
        "shortest_length_path": p_len,
        "shortest_length_stats": st_len,
        "constrained_min_risk_path": p_safe,
        "constrained_min_risk_stats": st_safe,
        "shortest_cost_path": p_cost,
        "shortest_cost_stats": st_cost,
        "notes": artifacts.notes,
    }


# Example usage:
result = run_one_od_routing(
    segments_panel_gdf=segments_df,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    year=2021, month=6,
    origin_lonlat=(13.3777, 52.5163),
    dest_lonlat=(13.4541, 52.5110),
    eps=0.10,
    alpha=1.0, beta=1.0, gamma=0.2,
)
print(result["graph_sanity"])
print("Shortest length:", result["shortest_length_stats"])
print("Constrained min-risk:", result["constrained_min_risk_stats"])



{'n_nodes': 3074, 'n_edges': 4395, 'seg_risk_min': 0.0, 'seg_risk_median': 0.0, 'seg_risk_max': 4000.0, 'length_median': 371.8095742150327, 'cost_median': 373.6283729421776, 'node_penalty_nonzero_share': 0.08964732650739476, 'graph_node_ids_attached': 2882, 'notes': 'Attached node_id to 2882/3074 graph nodes (snap<= 20.0m).'}
Shortest length: {'length_m': 6599.195578923747, 'seg_risk_sum': 0.0, 'node_penalty_sum': 0.0, 'cost_sum': 6599.195578923747}
Constrained min-risk: {'length_m': 6599.195578923747, 'seg_risk_sum': 0.0, 'node_penalty_sum': 0.0, 'cost_sum': 6599.195578923747}


## Evaluation loop over many OD pairs

In [9]:
# Extensive evaluation loop over MANY OD pairs

def _sample_reachable_od_pairs(
    G: nx.Graph,
    n_pairs: int,
    *,
    seed: int = 7,
    min_euclid_m: float = 1500.0,
    max_tries: int = 200000,
) -> list[tuple[tuple[float, float], tuple[float, float]]]:
    """
    Samples OD pairs as graph nodes (not lon/lat) ensuring:
      - O != D
      - straight-line distance between nodes >= min_euclid_m
      - O and D are connected (same component)

    Returns list of (source_node, target_node) where each node is the (x,y) NodeKey.
    """
    rng = random.Random(seed)
    nodes = list(G.nodes())
    if len(nodes) < 2:
        return []

    # Precompute component id for each node to ensure reachability
    comp_map = {}
    for cid, comp in enumerate(nx.connected_components(G.to_undirected())):
        for n in comp:
            comp_map[n] = cid

    pairs = []
    tries = 0
    while len(pairs) < n_pairs and tries < max_tries:
        tries += 1
        a = rng.choice(nodes)
        b = rng.choice(nodes)
        if a == b:
            continue
        if comp_map.get(a) != comp_map.get(b):
            continue

        dx = float(a[0] - b[0])
        dy = float(a[1] - b[1])
        if math.sqrt(dx * dx + dy * dy) < min_euclid_m:
            continue

        pairs.append((a, b))

    return pairs


def evaluate_many_od_pairs(
    *,
    segments_panel_gdf: gpd.GeoDataFrame,
    crossings_gdf: gpd.GeoDataFrame,
    junction_panel_gdf: gpd.GeoDataFrame,
    year: int,
    month: int,
    n_pairs: int = 200,
    eps: float = 0.10,
    # cost weights
    alpha: float = 1.0,
    beta: float = 1.0,
    gamma: float = 0.0,
    metric_epsg: int = 32633,
    seed: int = 7,
    min_euclid_m: float = 1500.0,
) -> pd.DataFrame:
    """
    Builds month graph once, samples many reachable OD pairs on the graph,
    and evaluates:
      - shortest-by-length
      - shortest-by-cost
      - constrained-min-risk route (risk subject to length constraint)

    Returns a DataFrame with per-OD statistics + summary columns suitable for paper plots.
    """
    artifacts = build_graph_with_costs_for_month(
        segments_panel_gdf,
        year,
        month,
        crossings_gdf=crossings_gdf,
        junction_panel_gdf=junction_panel_gdf,
        graph_cfg=GraphBuildConfig(metric_epsg=metric_epsg),
        cost_cfg=CostConfig(alpha=alpha, beta=beta, gamma=gamma),
        node_snap_m=20.0,
    )

    sanity = verify_graph_sanity(artifacts, expect_junction_penalties=(gamma != 0.0))
    G = artifacts.G

    # Sample OD pairs directly from graph nodes to avoid repeated lon/lat snapping and to ensure reachability
    od_pairs = _sample_reachable_od_pairs(G, n_pairs, seed=seed, min_euclid_m=min_euclid_m)
    if len(od_pairs) == 0:
        raise ValueError("Could not sample any reachable OD pairs. Check graph connectivity / size.")

    rows = []
    for i, (src, dst) in enumerate(od_pairs):
        # 1) shortest length
        p_len = shortest_path_by(G, src, dst, weight="length_m")
        st_len = route_stats(G, p_len)
        shortest_len = st_len["length_m"]
        max_len = (1.0 + eps) * shortest_len

        # 2) shortest cost
        p_cost = shortest_path_by(G, src, dst, weight="cost")
        st_cost = route_stats(G, p_cost)

        # 3) constrained min-risk (risk = seg_risk)
        p_safe = constrained_min_risk_route(G, src, dst, eps=eps, length_attr="length_m", risk_attr="seg_risk")
        st_safe = route_stats(G, p_safe)

        rows.append({
            "pair_idx": i,
            "src_x": float(src[0]),
            "src_y": float(src[1]),
            "dst_x": float(dst[0]),
            "dst_y": float(dst[1]),

            "shortest_len_m": float(st_len["length_m"]),
            "shortest_risk_sum": float(st_len["seg_risk_sum"]),
            "shortest_cost_sum": float(st_len["cost_sum"]),

            "costroute_len_m": float(st_cost["length_m"]),
            "costroute_risk_sum": float(st_cost["seg_risk_sum"]),
            "costroute_cost_sum": float(st_cost["cost_sum"]),

            "safe_len_m": float(st_safe["length_m"]),
            "safe_risk_sum": float(st_safe["seg_risk_sum"]),
            "safe_cost_sum": float(st_safe["cost_sum"]),

            "len_constraint_max_m": float(max_len),
            "safe_feasible": bool(st_safe["length_m"] <= max_len + 1e-6),
        })

    df = pd.DataFrame(rows)

    # Convenience + risk trade-offs relative to shortest distance route
    df["safe_extra_len_pct"] = (df["safe_len_m"] - df["shortest_len_m"]) / df["shortest_len_m"]
    df["safe_risk_reduction_pct"] = (df["shortest_risk_sum"] - df["safe_risk_sum"]) / df["shortest_risk_sum"].replace(0, np.nan)

    df["cost_extra_len_pct"] = (df["costroute_len_m"] - df["shortest_len_m"]) / df["shortest_len_m"]
    df["cost_risk_reduction_pct"] = (df["shortest_risk_sum"] - df["costroute_risk_sum"]) / df["shortest_risk_sum"].replace(0, np.nan)

    # Attach run metadata for reproducibility
    df.attrs["graph_sanity"] = sanity
    df.attrs["notes"] = artifacts.notes
    df.attrs["year"] = year
    df.attrs["month"] = month
    df.attrs["eps"] = eps
    df.attrs["alpha"] = alpha
    df.attrs["beta"] = beta
    df.attrs["gamma"] = gamma
    df.attrs["n_pairs_requested"] = n_pairs
    df.attrs["n_pairs_used"] = len(df)

    return df


# Example usage:
eval_df = evaluate_many_od_pairs(
    segments_panel_gdf=segments_df,
    crossings_gdf=crossings_gdf,
    junction_panel_gdf=junction_df,
    year=2021, month=6,
    n_pairs=300,
    eps=0.10,
    alpha=1.0, beta=1.0, gamma=0.2,
    seed=7,
    min_euclid_m=2000.0,
)
print(eval_df.attrs["graph_sanity"])
print(eval_df[["safe_extra_len_pct", "safe_risk_reduction_pct"]].describe())


{'n_nodes': 3074, 'n_edges': 4395, 'seg_risk_min': 0.0, 'seg_risk_median': 0.0, 'seg_risk_max': 4000.0, 'length_median': 371.8095742150327, 'cost_median': 373.6283729421776, 'node_penalty_nonzero_share': 0.08964732650739476, 'graph_node_ids_attached': 2882, 'notes': 'Attached node_id to 2882/3074 graph nodes (snap<= 20.0m).'}
       safe_extra_len_pct  safe_risk_reduction_pct
count          300.000000               272.000000
mean             0.040958                 0.825061
std              0.030359                 0.315402
min              0.000000                 0.000000
25%              0.013169                 0.797813
50%              0.040338                 1.000000
75%              0.065013                 1.000000
max              0.099990                 1.000000


## Loop routing over all months

In [ ]:
results = []

for (year, month), _ in segments_df.groupby(["year", "month"]):
    eval_df = evaluate_many_od_pairs(
        segments_panel_gdf=segments_df,
        crossings_gdf=crossings_gdf,
        junction_panel_gdf=junction_df,
        year=year,
        month=month,
        n_pairs=200,
        eps=0.10,
        alpha=1.0,
        beta=1.0,
        gamma=0.2,
    )
    eval_df["year"] = year
    eval_df["month"] = month
    results.append(eval_df)

all_months_eval = pd.concat(results, ignore_index=True)
